# **재정정보 AI 검색 알고리즘 경진대회 - RAG, **
> **RAG, **

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

---

💡 **NOTE**: 이 예제에서 사용한 모델 및 벡터 DB 

1. LLM : mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1
2. Embed Model : intfloat/multilingual-e5-large
3. Vector DB : FAISS

---

# Install

In [1]:
# 모델 가속화 및 메모리 관리
%pip install accelerate
%pip install bitsandbytes

%pip install 'autoawq>=0.1.7'  
%pip install datasets
%pip install torch==2.2.0
%pip install torchvision==0.17.0
%pip install transformers -U
%pip install sentence-transformers 

# 벡터 데이터베이스 관련 라이브러리
# %pip install faiss-cpu
%pip install faiss-gpu
 
# 데이터
%pip install tiktoken
%pip install pymupdf4llm 
%pip install pandas 

# langchain
%pip install langchain 
%pip install langchain-community 
%pip install langchain-huggingface
%pip install langchain-anthropic

%pip install python-dotenv 
%pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 139.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.5/435.5 kB 131.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 46.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 40.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 

# Import Library

In [2]:
import os
import re
import json
import sys
import unicodedata
import logging
import torch
import random
from transformers import set_seed
import nest_asyncio

import pandas as pd
from tqdm import tqdm
import pymupdf
import pymupdf4llm
from collections import Counter

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    AwqConfig,
    TextStreamer,
    Gemma2ForCausalLM,
)
from accelerate import Accelerator

# Langchain 관련
from langchain import hub
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.document_loaders import PDFPlumberLoader, PyMuPDFLoader, PyPDFLoader, UnstructuredPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain.schema import Document
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.storage import LocalFileStore
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


# Setup

In [3]:


# 비동기 이벤트 루프를 재설정하거나 중첩할 수 있도록 허용
nest_asyncio.apply()

# 로깅 설정
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

# CUDA 메모리 할당이 필요할 때마다 메모리 블록을 확장
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 시드 설정
seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)

In [17]:

class Config:
    def __init__(self, llm_name: str, embedding_name: str, concept: str) -> None:        
        # 모델별 설정 딕셔너리
        self.model_config = {
            "meta-llama/Meta-Llama-3.1-8B-Instruct": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "jjjguz/Llama-3.1-Korean-8B-Instruct-v1": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "rtzr/ko-gemma-2-9b-it": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "mindsignal/upstage-SOLAR-10.7B-Instruct-v1.0-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": torch.float16,
                "max_token": 512
            }
        }
        
        # Large Language Model
        self.llm_name = llm_name
        self.llm_config = self.model_config[self.llm_name]
        self.llm_obj = self.setup_llm()
        
        # 임베딩 모델 설정
        self.embedding_name = embedding_name
        self.embedding_obj = self.setup_embedding()
        
        # document
        self.chunk_size = 512
        self.chunk_overlap = 32
        self.bm25_w = 0.5
        self.faiss_w = 0.5
        
        # Data Path
        self.base_directory = "open/"
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        
        # Output Path
        self.output_dir = "test_results"
        self.output_csv_file = f"{self.llm_name.split('/')[1]}_{concept}_submission.csv"
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Langsmith 설정 (환경 변수)
        os.environ["LANGCHAIN_TRACING_V2"] = "true" # os.getenv("LANGCHAIN_TRACING_V2", "false")
        os.environ["LANGCHAIN_API_KEY"] = "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" # os.getenv("LANGCHAIN_API_KEY")
        os.environ["LANGCHAIN_PROJECT"] = "Docon-Financial-search" 
    
    def to_json(self):
        return json.dumps(self.__dict__)
    
    def get_quantization_config(self):
        """4-bit 양자화 설정을 반환하는 함수"""
        return BitsAndBytesConfig(
            load_in_4bit=True,  # 4-bit 양자화
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16  # 연산에 사용할 데이터 타입
        )
        
    def setup_embedding(self):
        """ 임베딩 모델 설정 """
        embed_id = self.embedding_name
        
        model_kwargs = {'device': 'cuda'}
        encode_kwargs = {'normalize_embeddings': True}
        embd = HuggingFaceEmbeddings(
            model_name=embed_id,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
        
        store = LocalFileStore("./cache/")
        
        # Cache Embedding 사용
        cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
            underlying_embeddings=embd, 
            document_embedding_cache=store, 
            namespace=embed_id
        )
        
        return cached_embeddings
    
    def setup_llm(self):
        """LLM 설정 및 파이프라인 구성"""
        model_id = self.llm_name
        
        # 토크나이저 로드 및 설정
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokenizer.use_default_system_prompt = False
        
        # 모델 로드 및 양자화 설정 적용
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            quantization_config=self.llm_config["quantization_config"],
            trust_remote_code=True
        )
        
        # 모델을 여러 GPU에 할당
        accelerator = Accelerator()
        model = accelerator.prepare(model)
        
        print(f"#### [ model ] ####\n{model}\n###################")
        
        # 스트리머를 설정하여 토큰이 생성될 때마다 출력
        streamer = TextStreamer(tokenizer)

        # HuggingFacePipeline 객체 생성
        text_generation_pipeline = pipeline(
            model=model,
            tokenizer=tokenizer,
            task="text-generation",
            return_full_text=False,
            max_new_tokens=450,
            streamer=streamer,
        )

        hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

        return hf

# 1

In [15]:
from langchain_core.runnables import ConfigurableField

class DataIngestionPipeline:
    def __init__(self, config:Config):
        self.config = config
        
    def normalize_path(self, path):
        """ Path 유니코드 정규화 """
        normalized_path = unicodedata.normalize('NFD', path)
        logger.debug(f"정규화된 경로: {normalized_path}")
        return normalized_path
    
    def preprocess_text(self, text: str) -> str:
        """ 텍스트 전처리 함수 """
        # 불필요한 공백 제거
        text = text.strip()
        # 마침표 뒤 및 "----" 전후의 줄바꿈을 제외한 모든 줄바꿈을 제거
        text = re.sub(r'(?<!\.)(?<!-----)(\n|\r\n)(?!-----)', ' ', text) # gemma
        # 공백 처리 및 불필요한 줄바꿈 제거
        text = re.sub(r'\s+', ' ', text)
        return text
    
    def process_pdf(self, file_path: str) -> list:
        """ PDF 파일 로드, 텍스트 추출 및 전처리 후 노드 생성 """
        logger.info(f"PDF 처리 중: {file_path}")
        md_content = pymupdf4llm.to_markdown(file_path)
        
        preprocess_content = self.preprocess_text(md_content)
                
        # headers_to_split_on = [
        #     ("#", "Header 1"),
        #     ("##", "Header 2"),
        #     ("###", "Header 3"),
        # ]
        
        # md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
        # md_chunks = md_header_splitter.split_text(md_content)
        
        # 텍스트 데이터를 문서 청크로 분할
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=self.config.chunk_size, chunk_overlap=self.config.chunk_overlap
        )
        chunks = text_splitter.split_text(preprocess_content)
        logger.info(f"텍스트를 {len(chunks)}개의 청크로 분할 완료.")
        
        return chunks
    
    def init_vector_db(self, df):
        """ PDF 데이터를 처리하고 벡터 데이터베이스 및 리트리버를 초기화 """
        pdf_databases = {}
        unique_paths = df['Source_path'].unique()
        
        for path in tqdm(unique_paths, desc="Processing PDFs"):
            # 경로 정규화 및 절대 경로 생성
            normalized_path = self.normalize_path(path)
            full_path = os.path.normpath(os.path.join(self.config.base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
            
            # pdf 제목 추출
            pdf_title = os.path.splitext(os.path.basename(full_path))[0]
            print(f"Processing {pdf_title}...")
            
            # PDF 처리 및 벡터 DB 생성
            chunks = self.process_pdf(full_path)

            vector_store = FAISS.from_texts(chunks, embedding=self.config.embedding_obj)    
            faiss_retriever = vector_store.as_retriever(search_kwargs={"k": 3}).configurable_fields(
                search_kwargs = ConfigurableField(
                    id='search_kwargs_faiss',
                    name='Search Kwargs',
                    description='The search kwargs to use',
                )
            )
            
            bm25_retriever = BM25Retriever.from_texts(chunks)
            bm25_retriever.k = 3
                        
            # 앙상블 리트리버 생성 (BM25 기반 + FAISS 임베딩 기반)
            ensemble_retriever = EnsembleRetriever(
                retrievers=[bm25_retriever, faiss_retriever],
                weights=[self.config.bm25_w, self.config.faiss_w],
                search_type="mmr",
            )

            # 결과 저장
            pdf_databases[pdf_title] = {
                'vector_store': vector_store,
                'retriever': ensemble_retriever
            }
            
        return pdf_databases
                


# Prompt

In [24]:
""" QA 프롬프트 템플릿 """

template = """ 
당신은 재정 정책 전문가입니다. 주어진 정보를 바탕으로, 명확하고 정확하게 답변해 해주세요.
답변을 생성할 때는 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 답변하세요.

주어진 정보는 다음과 같습니다. : 
{context}

질문 :
{question}

새로운 질문을 생성하지 마세요.
주어진 정보의 내용을 참고해서 질문에 정확한 답변을 생성하세요.
기호나 공백을 제거하고 깔끔하게 답변하세요.
"""
# financial_qa_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt_template),
#         ("human", "{input}"),
#     ]
# )
financial_qa_prompt = PromptTemplate.from_template(template) 

# QA 생성 모듈

In [22]:

class FISearch:
    def __init__(self, config: Config, prompts) -> None:
        self.config = config
        self.prompts = prompts

    def normalize_string(self, s):
        """ 유니코드 정규화 """
        return unicodedata.normalize('NFC', s)
    
    def format_docs(self, docs):
        # 문서의 페이지 내용을 이어붙여 반환합니다.
        return "\n\n".join(doc.page_content for doc in docs)

    def generate_answer(self, df, pdf_databases):
        """ DataFrame의 각 질문에 대한 답변 생성 후 저장 """
        results = []

        # DataFrame의 각 행에 대해 처리
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
            # 소스 문자열 정규화
            source = self.normalize_string(row['Source'])
            question = row['Question']

            # 정규화된 키로 데이터베이스 검색
            normalized_keys = {self.normalize_string(k): v for k, v in pdf_databases.items()}
            ensemble_retriever = normalized_keys[source]['retriever'] # 앙상블 리트리버

            # RAG 체인 정의
            rag_chain = (
                {"context": ensemble_retriever | self.format_docs, "question": RunnablePassthrough()}
                | self.prompts
                | self.config.llm_obj
                | StrOutputParser()
            )
            
            # # Retrieval Chain 생성
            # documents_chain = create_stuff_documents_chain(self.config.llm_obj, prompt=self.prompts)
            # rag_chain = create_retrieval_chain(ensemble_retriever, documents_chain) # EnsembleRetreiver + document chain
            
            # 답변 생성
            print(f"질문 : {question}")
            result = rag_chain.invoke(question)
            print(f"답변: {result}\n================================================")
            
            # 결과 저장
            results.append({
                "Source": row['Source'],
                "Source_path": row['Source_path'],
                "Question": question,
                "Answer": result
            })

        return results
    

---

# 최종 통합, 실행 및 저장

In [25]:

if __name__ == "__main__":
    # 초기 세팅
    config = Config(
        llm_name="mindsignal/upstage-SOLAR-10.7B-Instruct-v1.0-4bit-financesinfo-ver1", 
        embedding_name="intfloat/multilingual-e5-large",
        concept="financial_analysis"
    )
    
    # 데이터 불러오기
    df = pd.read_csv(config.test_csv_path)
    
    # 파이프라인 초기화
    dipipeline = DataIngestionPipeline(config)
    
    # 벡터 데이터베이스 초기화 - 메모리에 저장
    pdf_databases = dipipeline.init_vector_db(df)
    
    # 검색 및 답변 생성
    search_engine = FISearch(config, prompts=financial_qa_prompt)
    results = search_engine.generate_answer(df, pdf_databases)
    
    # 결과를 제출 양식에 맞게 저장
    submit_df = pd.read_csv(f"./open/sample_submission.csv")
    submit_df['Answer'] = [item['Answer'] for item in results]
    submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")
    
    # 결과를 CSV 파일로 저장
    submit_df.to_csv(os.path.join(config.output_dir, config.output_csv_file), encoding='UTF-8-sig', index=False)
    
    print("결과가 성공적으로 저장되었습니다.")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

#### [ model ] ####
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_lay

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...
INFO:__main__:PDF 처리 중: open/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
INFO:__main__:텍스트를 10개의 청크로 분할 완료.


Processing PDFs:  11%|█         | 1/9 [00:00<00:02,  3.72it/s]

...

답변: 
답변:
국가결산보고서는 '우발사항'으로 공시하고, 지방자치단체 회계기준은 '우발손실'로 정의합니다.<eos>
질문 : 우발부채란 무엇이며, 그 관리가 왜 중요한가?
<s>  
당신은 재정 정책 전문가입니다. 주어진 정보를 바탕으로, 명확하고 정확하게 답변해 해주세요.
답변을 생성할 때는 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 답변하세요.

주어진 정보는 다음과 같습니다. : 
GFSM2014와 동일한 우발부채 개념을 적용하고 있음 **GFSM2014는 우발부채를 크게 명시적(explicit) 우발부채와 암묵적(implicit) 우발부채로 구분** ‣ ‘명시적 우발부채’는 규정된 조건 발생시 경제적 가치의 지급을 요구하는 법적 또는 계약에 의한 금융협정 으로서, ‘(일회성) 보증’[18)]과 그 외에 ‘기타 명시적 우발부채’로 구분 - 명시적 우발부채의 보편적인 형태는 ‘보증’이지만 모든 보증이 우발부채로 분류되는 것은 아닌데, 예컨대 파

발생주의 기준을 적용한 국가결산보고서에서 우발부채를 보고하고 있으나, 용어 사용에 혼란이 있고 분류상 오류가 있는 등 체계적 관리 대안에 대한 논의가 필요한 시점 - 국제적으로 정부의 채무지속가능성분석(DSA)[4)]을 위한 기초자료로 활용되는 등 주요하게 인식되는 추세와 비교해 우리나라에서는 아직까지 체계적으로 다뤄지지 않고 있음 **FOCUS** **우발부채 관련 주요 쟁점은?** **FOCUS** ‣ 우발부채 개념 및 용어 사용의 혼란,

basis)와 차이가 있다. **2) 의제의무(擬制義務, constructive obligation)란 발표된 정부방침 또는 구체적이고 유효한 약속이나 과거의 실무관행 등을 통해 중앙관서 또는 기금이 특정 책임을** 부담한다는 것을 표명함으로써 상대방이 그 책임을 이행할 것이라는 정당한 기대를 가지게 되는 경우 발생하는 의무를 말한다(「충당부채, 우발부채, 우발자산 회계처리지침」, 2018). **3) 국제통화기금(IMF)은 재정통계 작성의 국제기준

Answering Questions:  99%|█████████▉| 97/98 [1:18:58<01:23, 83.04s/it]

답변: 
답변:
우발부채는 재정 정책 수행에 따른 예상 외의 지출로 인한 부채로, 다만 정부가 직접 대신해 지급할 의무가 아니기 때문에 보통은 부채 계산에서 제외되지만, 경우에 따라 국가채무에 포함되는 경우도 있음 - 우발부채의 관리가 중요한 이유는 우발부채가 포함하는 다양한 의제의무(constructive obligation) 중 일부는 정부에 대한 지급 의무로 전환될 수 있기 때문임
































































































































































































































































질문 : 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?
<s>  
당신은 재정 정책 전문가입니다. 주어진 정보를 바탕으로, 명확하고 정확하게 답변해 해주세요.
답변을 생성할 때는 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 답변하세요.

주어진 정보는 다음과 같습니다. : 
현재까지 총 2차례(2001년, 2014년)의 개정이 있었다. 특히, GFSM 2001은 전면 개정을 통해 현재의 발생주의 기준 GFS체계를** 구축하였으며, GFSM2014는 SNA, PSDS 등 다른 국제통계기준과 부합하도록 더 구체화했다. **18) 보증(guarantees)은 명시적 우발부채의 가장 보편적 형태이나 모든 보증이 우발부채에 해당하는 것은 아니다. 예컨대 ‘파생금융상품 형태의 보증’이나 ‘표준화된 보증** 제도하에서 설정하는 충당부채 형태의 보증’은 재정상태표에 부채로 기록한다.

것은 아닌데, 예컨대 파 생금융상품 형태의 보증[19)]과 표준

Answering Questions: 100%|██████████| 98/98 [1:20:31<00:00, 49.30s/it]

답변: 
답변:
보증은 타인의 부채 상환(예: 대출 상환) 불가능성에 대비한 위험 전가 목적으로 발생하며, 부채(예: 대출) 상환 불가능성이 발생하는 경우 보증인이 대신 상환해야 하는 부채로 재정상태표에 부채로 기록된다. 표준화 보증은 통상 아주 적은 금액에 대해 획일적 조건으로 대규모로 발행하는 보증으로, 수출(무역)신용 보증, 환보증, 다양한 종류의 보험(예금, 농작물, 자연재해 등), 농민융자, 모기지론, 학자금융자, 중소기업융자 등이 있다.<eos>
결과가 성공적으로 저장되었습니다.
